In [60]:
import pandas as pd
import scipy.stats as st

In [61]:
import hvplot.pandas
hvplot.extension('plotly')

In [62]:
dfjan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
dffeb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

# Question 1
Read the data for January. How many columns are there?

* [ ] 16
* [ ] 17
* [ ] 18
* [x] 19

In [63]:
dfjan.shape

(3066766, 19)

In [64]:
[*map(len, (df_train, df_val))]

[3008892, 2855630]

In [65]:
for df in (dfjan, dffeb):
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda x: x.total_seconds() / 60)

# Question 2

What's the standard deviation of the trips duration in January?

* [ ] 32.59
* [x] 42.59
* [ ] 52.59
* [ ] 62.59

In [66]:
dfjan.duration.std()

np.float64(42.59435124195458)

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* [ ] 90%
* [ ] 92%
* [ ] 95%
* [x] 98%

In [67]:
((dfjan.duration > 1) & (dfjan.duration <= 60)).mean()

np.float64(0.9811286547457485)

In [68]:
dfjan, dffeb = [
    *map(lambda df: df.query("duration > 1 & duration <= 60"), (dfjan, dffeb))
]


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* [ ] 2
* [ ] 155
* [ ] 345
* [x] 515
* [ ] 715

In [69]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

In [70]:
ohe = DictVectorizer()
X_train = ohe.fit_transform(
    dfjan[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
    )

In [71]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6017784 stored elements and shape (3008892, 515)>

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* [ ] 3.64
* [x] 7.64
* [ ] 11.64
* [ ] 16.64


In [72]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [73]:
lr = LinearRegression()
lr.fit(X_train, dfjan['duration'])

LinearRegression()

In [74]:
y_train_pred = lr.predict(X_train)

In [75]:
root_mean_squared_error(dfjan['duration'], y_train_pred)

7.648231384745714

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* [ ] 3.81
* [x] 7.81
* [ ] 11.81
* [ ] 16.81

In [76]:
X_val = ohe.transform(
    dffeb[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
)


In [77]:
y_val_pred = lr.predict(X_val)

In [78]:
root_mean_squared_error(dffeb['duration'], y_val_pred)

7.810095061511874